# Homework #2

**Set up the environment**

In [54]:
import numpy as np
import pandas as pd

**Dataset**

In [56]:
# The Laptops price dataset from Kaggle
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/laptops.csv"
filename = "laptops.csv"
urllib.request.urlretrieve(url, filename)

('laptops.csv', <http.client.HTTPMessage at 0x21bec3cc250>)

In [58]:
df = pd.read_csv("laptops.csv")
#df.head()

In [60]:
# Normalize the name of columns
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [62]:
df.head()

,laptop,status,brand,model,cpu,ram,storage,storage_type,gpu,screen,touch,final_price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,NaN,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,NaN,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,NaN,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,NaN,15.6,No,669.01


In [100]:
df_new = df[["ram", "storage", "screen", "final_price"]]
df_new.head()

,ram,storage,screen,final_price
0,8,512,15.6,1009.00
1,8,256,15.6,299.00
2,8,256,15.6,789.00
3,16,1000,15.6,1199.00
4,16,512,15.6,669.01


**Question 1** <br>
There's one column with missing values. What is it?

In [102]:
df_new.isnull().sum()

ram            0
storage        0
screen         4
final_price    0
dtype: int64

**Question 2** <br>
What's the median (50% percentile) for variable 'ram'?

In [104]:
df_new["ram"].median()

16.0

**Prepare and split the dataset** <br>
* Shuffle the dataset (the filtered one you created above), use seed 42. <br>
* Split your data in train/val/test sets, with 60%/20%/20% distribution.

In [106]:
n = len(df_new)
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

# n, n_val, n_test, n_train

np.random.seed(42)

idx = np.arange(n)
np.random.shuffle(idx)
#idx

In [108]:
df_train = df_new.iloc[idx[:n_train]]
df_val = df_new.iloc[idx[n_train:n_train + n_val]]
df_test = df_new.iloc[idx[n_train + n_val:]]

#df_train.head()

In [110]:
y_train = df_train.final_price.values
y_val = df_val.final_price.values
y_test = df_test.final_price.values

del df_train["final_price"]
del df_val["final_price"]
del df_test["final_price"]

**Question 3**
* We need to deal with missing values for the column from Q1.
* We have two options: fill it with 0 or with the mean of this variable.
* Try both options. For each, train a linear regression model without regularization using the code from the lessons.
* For computing the mean, use the training only!
* Use the validation dataset to evaluate the models and compare the RMSE of each option.
* Round the RMSE scores to 2 decimal digits using round(score, 2)
* Which option gives better RMSE? <br>

Options:

* With 0
* With mean
* Both are equally good

In [126]:
base =["ram", "storage", "screen"]
# Prepare X function (fill the missing values with 0)
def prepare_X_zero(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

# Prepare X function (fill missing values with the mean)
def prepare_X_mean(df, mean_value):
    df_num = df[base]
    df_num = df_num.fillna(mean_value)
    X = df_num.values
    return X

# Train Linear Regression
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0]) # Add intercept
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)

    return w[0], w[1:]

# RMSE function
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [128]:
# Prepare the training data (fill with 0)
X_train_zero = prepare_X_zero(df_train)
w0_zero, w_zero = train_linear_regression(X_train_zero, y_train)

#Make predictions on the training set
y_pred_train_zero = w0_zero + X_train_zero.dot(w_zero)

# Calculate RMSE for the training set
rmse_train_zero = rmse(y_train, y_pred_train_zero)

# For the validation set
X_val_zero = prepare_X_zero(df_val)
y_pred_val_zero = w0_zero + X_val_zero.dot(w_zero)

# Calculate RMSE for validation data
rmse_val_zero = rmse(y_val, y_pred_val_zero)
print(f"RMSE (filling with 0) - Train: {round(rmse_train_zero, 2)}, Validation: {round(rmse_val_zero, 2)}")

RMSE (filling with 0) - Train: 602.39, Validation: 597.36


In [130]:
# Calculate the mean of the "screen" column 
mean_screen_train = df_train["screen"].mean()

# Prepare the training data (fill with mean)
X_train_mean = prepare_X_mean(df_train, mean_screen_train)
w0_mean, w_mean = train_linear_regression(X_train_mean, y_train)

# Make predictions on the training set
y_pred_train_mean = w0_mean + X_train_mean.dot(w_mean)

# Calculate RMSE for the training data
rmse_train_mean = rmse(y_train, y_pred_train_mean)

# For the validation set
X_val_mean = prepare_X_mean(df_val, mean_screen_train)
y_pred_val_mean = w0_mean + X_val_mean.dot(w_mean)

# Calculate RMSE for validation data
rmse_val_mean = rmse(y_val, y_pred_val_mean)
print(f"RMSE (filling with mean) - Train: {round(rmse_train_mean, 2)}, Validation: {round(rmse_val_mean, 2)}")

RMSE (filling with mean) - Train: 602.18, Validation: 600.27


**Question 4**
* Now let's train a regularized linear regression.
* For this question, fill the NAs with 0.
* Try different values of r from this list: [0, 0.01, 0.1, 1, 5, 10, 100].
* Use RMSE to evaluate the model on the validation dataset.
* Round the RMSE scores to 2 decimal digits.
* Which r gives the best RMSE? <br>

If there are multiple options, select the smallest r. <br>

Options:

* 0
* 0.01
* 1
* 10
* 100

In [148]:
n = len(df_new)
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

#n, n_val, n_test, n_train

np.random.seed(42)

idx = np.arange(n)
np.random.shuffle(idx)
#idx

df_train = df_new.iloc[idx[:n_train]]
df_val = df_new.iloc[idx[n_train:n_train + n_val]]
df_test = df_new.iloc[idx[n_train + n_val:]]

y_train = df_train.final_price.values
y_val = df_val.final_price.values
y_test = df_test.final_price.values

del df_train["final_price"]
del df_val["final_price"]
del df_test["final_price"]

In [150]:
base =["ram", "storage", "screen"]
# Prepare X function (fill the missing values with 0)
def prepare_X_reg(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

# Regularized Linear Regression
def train_linear_regression_reg(X, y, r=0.0):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X]) # Add intercept

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0]) # Add regularization term
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

# RMSE function
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [152]:
# Prepare the training and validation data
X_train_reg = prepare_X_reg(df_train)
X_val_reg = prepare_X_reg(df_val)

r_values = [0, 0.01, 0.1, 1, 5, 10, 100]

best_r = None
best_rmse_val = float("inf")

for r in r_values:
    w0_reg, w_reg = train_linear_regression_reg(X_train_reg, y_train, r)

    # Make predictions on the training set
    y_pred_train_reg = w0_reg + X_train_reg.dot(w_reg)
    # Calculate RMSE for the training set
    rmse_train_reg = rmse(y_train, y_pred_train_reg)

    # Make predictions on the validation set
    y_pred_val_reg = w0_reg + X_val_reg.dot(w_reg)
    # Calculate RMSE for validation data
    rmse_val_reg = rmse(y_val, y_pred_val_reg)
    
    print(f"r = {r}: RMSE (Train): {round(rmse_train_reg, 2)}, RMSE (Validation): {round(rmse_val_reg, 2)}")

r = 0: RMSE (Train): 602.39, RMSE (Validation): 597.36
r = 0.01: RMSE (Train): 602.39, RMSE (Validation): 597.36
r = 0.1: RMSE (Train): 602.39, RMSE (Validation): 597.35
r = 1: RMSE (Train): 602.41, RMSE (Validation): 597.21
r = 5: RMSE (Train): 602.71, RMSE (Validation): 597.01
r = 10: RMSE (Train): 603.13, RMSE (Validation): 597.06
r = 100: RMSE (Train): 604.99, RMSE (Validation): 597.9


**Question 5**
* We used seed 42 for splitting the data. Let's find out how selecting the seed influences our score.
* Try different seed values: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9].
* For each seed, do the train/validation/test split with 60%/20%/20% distribution.
* Fill the missing values with 0 and train a model without regularization.
* For each seed, evaluate the model on the validation dataset and collect the RMSE scores.
* What's the standard deviation of all the scores? To compute the standard deviation, use np.std.
* Round the result to 3 decimal digits (round(std, 3))

What's the value of std?

* 19.176
* 29.176
* 39.176
* 49.176

Note: Standard deviation shows how different the values are. If it's low, then all values are approximately the same. If it's high, the values are different. If standard deviation of scores is low, then our model is stable.

In [197]:
base =["ram", "storage", "screen"]
# Prepare X function (fill the missing values with 0)
def prepare_X_zero(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

# Train Linear Regression
def train_linear_regression(X, y):
    ones = np.ones(X.shape[0]) # Add intercept
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)

    return w[0], w[1:]

# RMSE function
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [199]:
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = [] # Placeholder to store RMSEs for each seed

In [201]:
for seed in seeds:
    # Split data
    np.random.seed(seed)
    
    n = len(df_new)

    n_val = int(n * 0.2)
    n_train = n - n_val - n_test
    
    idx = np.arange(n)
    np.random.shuffle(idx)

    df_train = df_new.iloc[idx[:n_train]]
    df_val = df_new.iloc[idx[n_train:n_train + n_val]]

    # Prepare the data
    X_train_zero = prepare_X_zero(df_train)
    y_train = df_train.final_price.values

    X_val_zero = prepare_X_zero(df_val)
    y_val = df_val.final_price.values

    # Train linear regression model without regularization
    w0_zero, w_zero = train_linear_regression(X_train_zero, y_train)

    # Make predictions on the validation set
    y_pred_val_zero = w0_zero + X_val_zero.dot(w_zero)
#    y_pred_val_zero = np.expm1(w0_zero + X_val_zero.dot(w_zero))
    
    # Calculate RMSE for validation data
    rmse_val_zero = rmse(y_val, y_pred_val_zero)
    rmse_scores.append(rmse_val_zero)

In [203]:
# Compute the standard deviation of RMSEs
std_rmse = np.std(rmse_scores)
std_rmse_rounded = round(std_rmse, 3)

print(f"RMSE scores: {rmse_scores}")
print(f"Standard deviation of RMSE: {std_rmse_rounded}")

RMSE scores: [565.4520868770995, 636.7985423056726, 588.955869790796, 597.8148920012515, 571.962791511103, 573.238325661895, 647.3438328407194, 550.4398184485962, 587.3335036169963, 576.101792943313]
Standard deviation of RMSE: 29.176


**Question 6**
* Split the dataset like previously, use seed 9.
* Combine train and validation datasets.
* Fill the missing values with 0 and train a model with r=0.001.
* What's the RMSE on the test dataset?

Options:
* 598.60
* 608.60
* 618.60
* 628.60

In [276]:
base =["ram", "storage", "screen"]
# Prepare X function (fill the missing values with 0)
def prepare_X_zero(df):
    df_num = df[base]
    df_num = df_num.fillna(0)
    X = df_num.values
    return X

# Regularized Linear Regression
def train_linear_regression_reg(X, y, r):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X]) # Add intercept

    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0]) # Add regularization term
    XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]

# RMSE function
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [278]:
n = len(df_new)
n_val = int(n * 0.2)
n_test = int(n * 0.2)
n_train = n - n_val - n_test

#n, n_val, n_test, n_train

np.random.seed(9)
idx = np.arange(n)
np.random.shuffle(idx)
#idx

df_train = df_new.iloc[idx[:n_train]]
df_val = df_new.iloc[idx[n_train:n_train + n_val]]
df_test = df_new.iloc[idx[n_train + n_val:]]

# Combine train and validation sets 
df_train_combined = pd.concat([df_train, df_val])

# Prepare the target
y_train_combined = df_train_combined.final_price.values
y_test = df_test.final_price.values

del df_train_combined["final_price"]
del df_test["final_price"]

In [280]:
# Prepare the training data (fill with 0)
X_train_zero = prepare_X_zero(df_train_combined)
X_test_zero = prepare_X_zero(df_test)

# Train regularized linear regression model with r = 0.001
w0_zero, w_zero = train_linear_regression_reg(X_train_zero, y_train_combined, 0.001)

#Make predictions on the test set
y_pred_test_zero = w0_zero + X_test_zero.dot(w_zero)

# Calculate RMSE for the test set
rmse_test_zero = rmse(y_test, y_pred_test_zero)

print(f"RMSE on the test set: {round(rmse_test_zero, 2)}")

RMSE on the test set: 608.61
